<a href="https://colab.research.google.com/github/Mxyzpltks/Sentimental-Analysis/blob/main/sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install kaggle

In [ ]:
# Create a directory for Kaggle API credentials
!mkdir -p ~/.kaggle

# Copy the Kaggle API key file to the proper location
!cp /content/drive/MyDrive/sentimental-analysis/kaggle.json ~/.kaggle/

# Set permissions for the file to ensure it's private
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d kazanova/sentiment140 -p /content/drive/MyDrive/sentimental-analysis

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!kaggle datasets list

ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                          Student Performance Factors                          94KB  2024-09-02 10:53:57          19836        376  1.0              
abdulszz/spotify-most-streamed-songs                             Spotify Most Streamed Songs                          60KB  2024-09-07 18:23:14           6042         78  1.0              
valakhorasani/mobile-device-usage-and-user-behavior-dataset      Mobile Device Usage and User Behavior Dataset        11KB  2024-09-28 20:21:12            720         27  1.0              
owm4096/laptop-prices                                  

In [ ]:
# from  zipfile import ZipFile
# dataset="/content/drive/MyDrive/sentimental-analysis/sentiment140.zip"
# with ZipFile(dataset, "r") as zip:
#     zip.extractall("/content/drive/MyDrive/sentimental-analysis/")
#     print("Dataset has been extracted!!!")

!unzip -qq /content/drive/MyDrive/sentimental-analysis/sentiment140.zip

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data processing

In [ ]:
twitter_data= pd.read_csv("/content/drive/MyDrive/sentimental-analysis/training.1600000.processed.noemoticon.csv",encoding="ISO-8859-1")

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head(10)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
6,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
8,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
9,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


In [ ]:
coloumn_names=["target","id","data","flag ","user " ,"text"]
twitter_data= pd.read_csv("/content/drive/MyDrive/sentimental-analysis/training.1600000.processed.noemoticon.csv",encoding="ISO-8859-1")

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
coloumn_names = ["target", "id", "data", "flag", "user", "text"]
twitter_data = pd.read_csv("/content/drive/MyDrive/sentimental-analysis/training.1600000.processed.noemoticon.csv", names=coloumn_names, encoding="ISO-8859-1")


In [ ]:
 twitter_data.isnull().sum()#to find the null values igno ideaig thats its for

,0
0,0
1467810369,0
Mon Apr 06 22:19:45 PDT 2009,0
NO_QUERY,0
_TheSpecialOne_,0
"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D",0


In [ ]:
#the following code is for count or check the number of data the positive and negative tweets
#the 0 is negitive tweet 4 is positive tweet
twitter_data["target"].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
#Changing the above 4 to 1
twitter_data.replace({"target": {4: 1}}, inplace=True)


In [ ]:
twitter_data["target"].value_counts()


,count
target,
0,800000
1,800000


STEMMING


In [ ]:
# STEMMIGN  a process of reducing the words to its root form the below words are some examples
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content=re.sub("[^a-zA-Z] "," ",content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content=" ".join(stemmed_content)
    return stemmed_content


In [ ]:
  twitter_data["stemmed_content"]=twitter_data["text"].apply(stemming)
  twitter_data.to_csv('/content/drive/MyDrive/sentimental-analysis/twitter_data_stemmed.csv', index_col=None)


KeyboardInterrupt: 

In [ ]:
twitter_data_stemmed= pd.read_csv('/content/drive/MyDrive/sentimental-analysis/twitter_data_stemmed.csv', index_col=None)


In [ ]:
twitter_data_stemmed.head(5)

,target,id,data,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",@switchfoot http://twitpic.com/2y1zl awww that...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset can't updat facebook text it.. might cri...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichan dive mani time ball manag save 50 re...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",@nationwideclass behav i'm mad can't see


In [ ]:
print(twitter_data_stemmed["stemmed_content"])

0          @switchfoot http://twitpic.com/2y1zl awww that...
1          upset can't updat facebook text it.. might cri...
2          @kenichan dive mani time ball manag save 50 re...
3                            whole bodi feel itchi like fire
4                   @nationwideclass behav i'm mad can't see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb.com cool hear old walt interview Ã¢Â h...
1599997                         readi mojo makeov ask detail
1599998    happi 38th birthday boo alll time!! tupac amar...
1599999    happi #charitytuesday @thenspcc @sparkschar @s...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data_stemmed["target"])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
twitter_data_stemmed['stemmed_content'] = twitter_data_stemmed['stemmed_content'].fillna('')


In [ ]:
X= twitter_data_stemmed["stemmed_content"]
Y=twitter_data_stemmed["target"]

In [ ]:
print(X)

0          @switchfoot http://twitpic.com/2y1zl awww that...
1          upset can't updat facebook text it.. might cri...
2          @kenichan dive mani time ball manag save 50 re...
3                            whole bodi feel itchi like fire
4                   @nationwideclass behav i'm mad can't see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb.com cool hear old walt interview â htt...
1599997                         readi mojo makeov ask detail
1599998    happi 38th birthday boo alll time!! tupac amar...
1599999    happi #charitytuesday @thenspcc @sparkschar @s...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(Y)

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)
"""X: Represents the feature matrix (the input data or independent variables).
Y: Represents the target vector (the labels or dependent variable)"""

'X: Represents the feature matrix (the input data or independent variables).\nY: Represents the target vector (the labels or dependent variable)'

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

1570269                          watch saw iv drink lil wine
1273074                                    @hatermagazin i'm
88479      even though favourit drink think vodka coke wi...
254604                  think hand got burnt sun today, hurt
667941     took mazi dr.for shot today come find ear infe...
                                 ...                        
941805                                  @threewink cheers...
1007131    vote livewir play live smith tomorrow night su...
1460311                               eager monday afternoon
929226     hope everyon mother great day can't wait hear ...
526253                      love wake folger bad voic deeper
Name: stemmed_content, Length: 1280000, dtype: object


In [ ]:
print(X_test)

['@mmangen fine much time chat twitter hubbi back summer &amp tend domin free time'
 'ah may show w ruth kim &amp geoffrey sanhueza'
 '@ishatara mayb bay area thang dammit!' ...
 '@destini4 nevertheless hooray 470 member wonder safe trip' 'feel well'
 '@supersandro !!! thank']


In [ ]:
twitter_data_stemmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   target           1600000 non-null  int64 
 1   id               1600000 non-null  int64 
 2   data             1600000 non-null  object
 3   flag             1600000 non-null  object
 4   user             1600000 non-null  object
 5   text             1600000 non-null  object
 6   stemmed_content  1600000 non-null  object
dtypes: int64(2), object(5)
memory usage: 85.4+ MB


In [ ]:
twitter_data_stemmed.isnull().sum()

,0
target,0
id,0
data,0
flag,0
user,0
text,0
stemmed_content,0


In [ ]:
twitter_data_stemmed.describe()

,target,id
count,1600000.0,1.600000e+06
mean,0.5,1.998818e+09
std,0.5,1.935761e+08
min,0.0,1.467810e+09
25%,0.0,1.956916e+09
50%,0.5,2.002102e+09
75%,1.0,2.177059e+09
max,1.0,2.329206e+09


In [ ]:
print(type(X_train))


<class 'numpy.ndarray'>


In [ ]:
# Check for missing values in the array
missing_count = np.sum([x is None or x == '' for x in X_train])
print(f"Number of missing values in X_train: {missing_count}")



Number of missing values in X_train: 0


In [ ]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
def detect(text):
  vectorizer = TfidfVectorizer()
  text = vectorizer.transform(text)

  return text

In [ ]:
print(X_train)

  (0, 532062)	0.272554884427638
  (0, 439619)	0.3585049844166932
  (0, 246036)	0.5252041070963219
  (0, 161655)	0.37629847055453536
  (0, 302726)	0.42194283114115794
  (0, 539049)	0.4491239977100762
  (1, 219013)	1.0
  (2, 161655)	0.4596464002033251
  (2, 178705)	0.1894135255192932
  (2, 499217)	0.18668087921816898
  (2, 184529)	0.29074148843649084
  (2, 498254)	0.32077390605256606
  (2, 528574)	0.32893274905823794
  (2, 125462)	0.31296319906891423
  (2, 539450)	0.33563423495836425
  (2, 339286)	0.24126161183427716
  (2, 501293)	0.1522483991239389
  (2, 238034)	0.16164645048291976
  (2, 207927)	0.1876258370366675
  (2, 187639)	0.2025108830989988
  (2, 364604)	0.16747523900603187
  (3, 498254)	0.28994476936054947
  (3, 216794)	0.44647083163571083
  (3, 209093)	0.2780160286287751
  (3, 100701)	0.5189402542431487
  :	:
  (1279996, 529061)	0.2688954517814905
  (1279996, 479870)	0.21772853250068736
  (1279996, 460496)	0.3559543462424634
  (1279996, 306251)	0.5371015554925388
  (1279996, 306

In [ ]:
print(X_test)

  (0, 54061)	0.17118130595536488
  (0, 72561)	0.16177326055165073
  (0, 114003)	0.2673339615119238
  (0, 157965)	0.36910495704962865
  (0, 187699)	0.2544238334108721
  (0, 194140)	0.23719540742086823
  (0, 230749)	0.2836692704465394
  (0, 344273)	0.4515147937047051
  (0, 353594)	0.17748255714254804
  (0, 477846)	0.21936183433534331
  (0, 490006)	0.34752541484343263
  (0, 501293)	0.3147229740913689
  (0, 514656)	0.17902320321926565
  (1, 44265)	0.30399480836208426
  (1, 54061)	0.20821424594665175
  (1, 202189)	0.5714146731177951
  (1, 282852)	0.4070732819545278
  (1, 327952)	0.2840923671284761
  (1, 432039)	0.4818916829327709
  (1, 452728)	0.24306898526504928
  (2, 62572)	0.3531860406495728
  (2, 76175)	0.37903231605173976
  (2, 139266)	0.3629427319713099
  (2, 244058)	0.588213814848275
  (2, 328042)	0.25720502008796964
  :	:
  (319995, 161738)	0.26048823894304113
  (319995, 213172)	0.23829126026777359
  (319995, 278189)	0.2307888592580303
  (319995, 300332)	0.22143645220813496
  (31999

Train the machine learning model

In [ ]:
model= LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(Y_train,X_train_prediction)

In [ ]:
print("the accuracy  on the training data is ",training_data_accuracy)

the accuracy  on the training data is  0.808184375


In [ ]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(Y_test,X_test_prediction)

In [ ]:
print("the accuracy  on the testing data is ",test_data_accuracy)

the accuracy  on the testing data is  0.7807125


training data is  80.81%

 testing data is  78.07%

 Saving the trained model



In [ ]:
import pickle

In [ ]:

fl="/content/drive/MyDrive/sentimental-analysis/trained_model.sav"
pickle.dump(model,open(fl,"wb"))

USING THE MODEL FOR FUTURE PREDICTION

In [ ]:
loaded_model = pickle.load(open("/content/drive/MyDrive/sentimental-analysis/trained_model.sav", "rb"))


In [ ]:
X_new=X_test[100]
print(X_new)

prediction=loaded_model.predict(X_new)
print(prediction)



if prediction[0]==0:
  print("NEGITIVE TWEET")
else:
  print("POSITIVE TWEET")

  (0, 63416)	0.2853846049109445
  (0, 75630)	0.3641524655331219
  (0, 93219)	0.3001059018888088
  (0, 146477)	0.21551030746043578
  (0, 182031)	0.29701677501229656
  (0, 202852)	0.11208425412515215
  (0, 225892)	0.23674776661803332
  (0, 243163)	0.2722017294580923
  (0, 350116)	0.21805760820301195
  (0, 362576)	0.3242639230620404
  (0, 422081)	0.31558212403528896
  (0, 424569)	0.23172166569849637
  (0, 434509)	0.185359540398046
  (0, 476791)	0.17361390515780337
  (0, 479681)	0.21916219116319996
[0]
NEGITIVE TWEET


In [ ]:
# new_sentance = "Hey it is a good day"
# new_sentance = detect(new_sentance)
prediction = loaded_model.predict(Y_test[100])

if prediction == 0:
  print("negative tweet")
else:
  print("positive tweet")


ValueError: Expected 2D array, got scalar array instead:
array=0.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
X_new=X_test[99]
print(Y_test[99])

prediction=loaded_model.predict(X_new)
print(prediction)



if prediction[0]==0:
  print("NEGITIVE TWEET")
else:
  print("POSITIVE TWEET")



KeyError: 99